In [32]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from tqdm.notebook import tqdm
from datetime import datetime
import sys
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
# from torch.optim.lr_scheduler import _LRScheduler
import torch.optim as optim

In [33]:
sys.path.append('..')
from gen.fishs_config import fishs

fishs = list(map(str.capitalize, fishs))

In [34]:
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        del df[each]
    return df

def one_hot_phenomenon(df, col):
    keys = set()
    for _, row in df.iterrows():
        phenomenon = row[col]
        for key in phenomenon.split('.'):
            keys.add(key)
    result = {key: [] for key in keys}
    for _, row in df.iterrows():
        phenomenon = row[col]
        for key in keys:
            if key in phenomenon.split('.'):
                result[key].append(1)
            else:
                result[key].append(0)
    for key in result:
        df[key] = result[key]
    del result
    del df[col]
    return df
            

def get_time_idx(row):
    try:
        return datetime(day=row['day'], month=row['month'], year=2019).timetuple().tm_yday
    except Exception:
        return -1

In [35]:
data = pd.read_csv(r'..\time_multiple_forecasts0.csv', sep=';')
data

,time,pressure,temperature,wind,gust,wind_direction,humidity,phenomenon,uv_index,moon_direction,...,Красноперка,Налим,Густера,Амур,Ерш,Сазан,Подуст,Толстолобик,Вобла,Хариус
0,0,749,-6,2,5,Ю,54,пасмурно,0,1,...,0.051212,0.635556,0.371429,0.0,0.485734,0.025809,0.0,0.0,0.055714,0.033183
1,3,749,-4,3,9,ЮЗ,55,пасмурно,0,1,...,0.128030,0.635556,0.417857,0.0,0.485734,0.029035,0.0,0.0,0.074286,0.044244
2,6,749,-4,3,10,ЮЗ,54,пасмурно,0,1,...,0.281667,0.577778,0.510714,0.0,0.441576,0.035487,0.0,0.0,0.204286,0.121670
3,9,749,-3,3,6,ЮЗ,51,пасмурно,0,1,...,0.281667,0.462222,0.510714,0.0,0.353261,0.035487,0.0,0.0,0.204286,0.121670
4,12,750,-1,5,13,ЮЗ,52,облачно,2,1,...,0.256061,0.231111,0.464286,0.0,0.309103,0.032261,0.0,0.0,0.185714,0.110609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244147,9,753,-6,0,7,ЮВ,41,пасмурно,0,-1,...,0.198611,0.121333,0.582214,0.0,0.576522,0.000000,0.0,0.0,0.247619,0.103645
244148,12,753,-3,0,5,В,38,пасмурно,2,-1,...,0.180556,0.060667,0.529286,0.0,0.504457,0.000000,0.0,0.0,0.225108,0.094223
244149,15,753,-3,0,8,ЮВ,36,облачно,1,-1,...,0.180556,0.091000,0.529286,0.0,0.504457,0.000000,0.0,0.0,0.225108,0.094223
244150,18,752,-4,2,8,В,28,пасмурно,0,-1,...,0.198611,0.136500,0.582214,0.0,0.648587,0.000000,0.0,0.0,0.202597,0.084800


In [36]:
time_idxs = []
for i, row in data.iterrows():
    try:
        time_idxs.append(get_time_idx(row))
    except Exception as e:
        print(e)
        print(row)
        break

In [37]:
data['day_year'] = time_idxs
data = one_hot(data, ['wind_direction', 'moon_direction', 'time'])
data = one_hot_phenomenon(data, 'phenomenon')
del data['day']
data


,pressure,temperature,wind,gust,humidity,uv_index,moon,month,Щука,Судак,...,снег,облачно,пасмурно,сильный снег,гроза,дождь,небольшой дождь,снег с дождём,сильный дождь,небольшой снег
0,749,-6,2,5,54,0,97,1,0.035846,0.357500,...,0,0,1,0,0,0,0,0,0,0
1,749,-4,3,9,55,0,97,1,0.179228,0.325000,...,0,0,1,0,0,0,0,0,0,0
2,749,-4,3,10,54,0,97,1,0.430147,0.357500,...,0,0,1,0,0,0,0,0,0,0
3,749,-3,3,6,51,0,97,1,0.394301,0.325000,...,0,0,1,0,0,0,0,0,0,0
4,750,-1,5,13,52,2,97,1,0.358456,0.325000,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244147,753,-6,0,7,41,0,79,12,0.499449,0.357762,...,0,0,1,0,0,0,0,0,0,0
244148,753,-3,0,5,38,2,79,12,0.454044,0.357762,...,0,0,1,0,0,0,0,0,0,0
244149,753,-3,0,8,36,1,79,12,0.408640,0.321986,...,0,1,0,0,0,0,0,0,0,0
244150,752,-4,2,8,28,0,79,12,0.522151,0.357762,...,0,0,1,0,0,0,0,0,0,0


In [38]:
y = data[fishs]
for fish in fishs:
    del data[fish]
data

,pressure,temperature,wind,gust,humidity,uv_index,moon,month,day_year,wind_direction_В,...,снег,облачно,пасмурно,сильный снег,гроза,дождь,небольшой дождь,снег с дождём,сильный дождь,небольшой снег
0,749,-6,2,5,54,0,97,1,2,0,...,0,0,1,0,0,0,0,0,0,0
1,749,-4,3,9,55,0,97,1,2,0,...,0,0,1,0,0,0,0,0,0,0
2,749,-4,3,10,54,0,97,1,2,0,...,0,0,1,0,0,0,0,0,0,0
3,749,-3,3,6,51,0,97,1,2,0,...,0,0,1,0,0,0,0,0,0,0
4,750,-1,5,13,52,2,97,1,2,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244147,753,-6,0,7,41,0,79,12,364,0,...,0,0,1,0,0,0,0,0,0,0
244148,753,-3,0,5,38,2,79,12,364,1,...,0,0,1,0,0,0,0,0,0,0
244149,753,-3,0,8,36,1,79,12,364,0,...,0,1,0,0,0,0,0,0,0,0
244150,752,-4,2,8,28,0,79,12,364,1,...,0,0,1,0,0,0,0,0,0,0


In [39]:
mean = data.mean()
std = data.std()
torch.save({'mean': mean, 'std': std}, 'scaler.pth')
data = (data - mean) / std
data

,pressure,temperature,wind,gust,humidity,uv_index,moon,month,day_year,wind_direction_В,...,снег,облачно,пасмурно,сильный снег,гроза,дождь,небольшой дождь,снег с дождём,сильный дождь,небольшой снег
0,0.294240,-1.096611,-0.247756,-1.076675,0.633557,-0.855887,1.550543,-1.601949,-1.712923,-0.330662,...,-0.119668,-0.542988,1.597115,-0.07654,-0.057766,-0.146777,-0.163159,-0.091497,-0.119333,-0.142423
1,0.294240,-0.935355,0.194191,0.331554,0.691099,-0.855887,1.550543,-1.601949,-1.712923,-0.330662,...,-0.119668,-0.542988,1.597115,-0.07654,-0.057766,-0.146777,-0.163159,-0.091497,-0.119333,-0.142423
2,0.294240,-0.935355,0.194191,0.683611,0.633557,-0.855887,1.550543,-1.601949,-1.712923,-0.330662,...,-0.119668,-0.542988,1.597115,-0.07654,-0.057766,-0.146777,-0.163159,-0.091497,-0.119333,-0.142423
3,0.294240,-0.854726,0.194191,-0.724618,0.460933,-0.855887,1.550543,-1.601949,-1.712923,-0.330662,...,-0.119668,-0.542988,1.597115,-0.07654,-0.057766,-0.146777,-0.163159,-0.091497,-0.119333,-0.142423
4,0.325759,-0.693470,1.078084,1.739783,0.518474,1.037646,1.550543,-1.601949,-1.712923,-0.330662,...,-0.119668,1.841652,-0.626127,-0.07654,-0.057766,-0.146777,-0.163159,-0.091497,-0.119333,-0.142423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244147,0.420319,-1.096611,-1.131649,-0.372560,-0.114482,-0.855887,0.939332,1.588937,1.717232,-0.330662,...,-0.119668,-0.542988,1.597115,-0.07654,-0.057766,-0.146777,-0.163159,-0.091497,-0.119333,-0.142423
244148,0.420319,-0.854726,-1.131649,-1.076675,-0.287106,1.037646,0.939332,1.588937,1.717232,3.024220,...,-0.119668,-0.542988,1.597115,-0.07654,-0.057766,-0.146777,-0.163159,-0.091497,-0.119333,-0.142423
244149,0.420319,-0.854726,-1.131649,-0.020503,-0.402189,0.090880,0.939332,1.588937,1.717232,-0.330662,...,-0.119668,1.841652,-0.626127,-0.07654,-0.057766,-0.146777,-0.163159,-0.091497,-0.119333,-0.142423
244150,0.388799,-0.935355,-0.247756,-0.020503,-0.862521,-0.855887,0.939332,1.588937,1.717232,3.024220,...,-0.119668,-0.542988,1.597115,-0.07654,-0.057766,-0.146777,-0.163159,-0.091497,-0.119333,-0.142423


In [40]:
torch.save(list(data.columns), 'features.keys')

In [41]:
class FishDataset(Dataset):
    def __init__(self, X, y, step_size=3):
        self.X = X
        self.y = y
        self.step_size = step_size
        self.steps = []
        
    def init_steps(self):
        steps = []
        for i in tqdm(range(0, len(self.X) - self.step_size)):
            step = list(self.X['day_year'][i:i+self.step_size * 8] * std['day_year'] + mean['day_year'])
            flag = True
            if len(step) != self.step_size * 8:
                continue
            for j in range(0, len(step) - 8, 8):
                if step[j] + 1 != step[j + 8]:
                    flag = False
                    break
            if flag:
                steps.append((i, i + self.step_size * 8))
        return steps
        
    def __len__(self):
        return len(self.steps)
    
    def __getitem__(self, idx):
        return self.X[self.steps[idx][0]:self.steps[idx][1]].values, self.y[self.steps[idx][0]:self.steps[idx][1]].values

In [42]:
steps = FishDataset(data, y, 3).init_steps()
mask = np.random.rand(len(data)) < 0.8
train_dataset = FishDataset(data, y, 3)
train_dataset.steps = [steps[i] for i in range(len(steps)) if mask[i]]
test_dataset = FishDataset(data, y, 3)
test_dataset.steps = [steps[i] for i in range(len(steps)) if not mask[i]]

In [43]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32, pin_memory=True)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=32, pin_memory=True)

In [52]:
for data, targets in train_dataloader:
    print(data.shape)
    print(targets.shape)
    break

torch.Size([32, 24, 40])
torch.Size([32, 24, 29])


In [63]:
class FCNBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim, bias=True)
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        return self.relu(self.fc(inputs))

class FCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.start_block = FCNBlock(input_dim, hidden_dim)
        self.blocks = nn.Sequential()
        for _ in range(layer_dim):
            self.blocks.add_module('block_{}'.format(_), FCNBlock(hidden_dim, hidden_dim))
        self.final_block = FCNBlock(hidden_dim, output_dim)
        
    def forward(self, inputs):
        return self.final_block(self.blocks(self.start_block(inputs)))
    

class LSTM(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t for t in (h0, c0)]
    
class MainModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.fcn = FCN(config['fcn_input'], config['fcn_hidden'], config['fcn_layers'], config['fcn_output'])
        self.embedder = FCN(config['emb_input'], config['emb_hidden'], config['emb_layers'], config['emb_output'])
        self.lstm = LSTM(config['lstm_input'], config['lstm_hidden'], config['lstm_layers'], config['lstm_output'])
        self.fc = nn.Linear(config['fcn_output'] + config['lstm_output'], config['output'])
    
    def forward(self, x,):
        fcn_output = self.fcn(x.view(x.size(0), -1))
        embeddings = self.embedder(x.view(-1, x.shape[-1])).reshape(x.shape[0], x.shape[1], -1)[:, :-1, :]
#         print(embeddings.shape)
        lstm_output = self.lstm(embeddings)
        output = self.fc(torch.cat((fcn_output, lstm_output), dim=1))
        return output, fcn_output, lstm_output
    
    
    
class MainLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.loss = nn.MSELoss()
        self.fcn_loss = nn.MSELoss()
        self.lstm_loss = nn.MSELoss()
        
    def forward(self, x, x_fcn, x_lstm, targets):
        loss = 2 * self.loss(x, targets) + self.fcn_loss(x_fcn, targets) + self.lstm_loss(x_lstm, targets)
        return loss

In [81]:
model_config = {
    'fcn_input': 24 * 40,
    'fcn_hidden': 128,
    'fcn_layers': 5,
    'fcn_output': 29,
    'emb_input': 40,
    'emb_hidden': 64,
    'emb_layers': 3,
    'emb_output': 29,
    'lstm_input': 29,
    'lstm_hidden': 64,
    'lstm_layers': 3,
    'lstm_output': 29,
    'output': 29
}

model = MainModel(model_config)
model

MainModel(
  (fcn): FCN(
    (start_block): FCNBlock(
      (fc): Linear(in_features=960, out_features=128, bias=True)
      (relu): ReLU()
    )
    (blocks): Sequential(
      (block_0): FCNBlock(
        (fc): Linear(in_features=128, out_features=128, bias=True)
        (relu): ReLU()
      )
      (block_1): FCNBlock(
        (fc): Linear(in_features=128, out_features=128, bias=True)
        (relu): ReLU()
      )
      (block_2): FCNBlock(
        (fc): Linear(in_features=128, out_features=128, bias=True)
        (relu): ReLU()
      )
      (block_3): FCNBlock(
        (fc): Linear(in_features=128, out_features=128, bias=True)
        (relu): ReLU()
      )
      (block_4): FCNBlock(
        (fc): Linear(in_features=128, out_features=128, bias=True)
        (relu): ReLU()
      )
    )
    (final_block): FCNBlock(
      (fc): Linear(in_features=128, out_features=29, bias=True)
      (relu): ReLU()
    )
  )
  (embedder): FCN(
    (start_block): FCNBlock(
      (fc): Linear(in_fea

In [82]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
loss_f = MainLoss()

In [83]:
def train(model, dataloader, optimizer, loss_f, log_interval, epoch, device='cpu'):
    running_loss = []
    for i, data in enumerate(dataloader):
        x, y = data
        targets = y[:, -1, :]
#         y = y[:, :-1, :]
#         x = x
        x = x.to(device).float()
#         y = y.to(device).float()
        targets = targets.to(device).float()
        output, fcn_output, lstm_output = model(x)
        loss = loss_f(output, fcn_output, lstm_output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_value = loss.item()
        if i % log_interval == 0 and len(running_loss) > 0:
            print('Train: Epoch {} [{:.2f}] Loss {:.2f} [{:.2f}]'.format(epoch, i / len(dataloader) * 100, loss_value, np.mean(running_loss)))
            running_loss = []
        running_loss.append(loss_value)
 
@torch.no_grad()
def validate(model, dataloader, loss_f, epoch, device='cpu'):
    running_loss = []
    for data in tqdm(dataloader):
        x, y = data
        targets = y[:, -1, :]
#         y = y[:, :-1, :]
#         x = x.view(x.size(0), -1)
        x = x.to(device).float()
#         y = y.to(device).float()
        targets = targets.to(device).float()
        output, fcn_output, lstm_output = model(x)
        loss = loss_f(output, fcn_output, lstm_output, targets)
        loss_value = loss.item()
        running_loss.append(loss_value)
    print('Test: Epoch {} Loss {:.2f}'.format(epoch, np.mean(running_loss)))

In [84]:
num_epochs = 15
log_interval = 100
device = 'cpu'
model.to(device)
loss_f.to(device)

for epoch in range(num_epochs):
    lr_scheduler.step()
    model.train()
    train(model, train_dataloader, optimizer, loss_f, log_interval, epoch, device)
    model.eval()
    validate(model, test_dataloader, loss_f, epoch, device)

C:\Users\Xiaomi\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: Epoch 0 [1.76] Loss 0.20 [0.29]
Train: Epoch 0 [3.52] Loss 0.13 [0.17]
Train: Epoch 0 [5.28] Loss 0.12 [0.15]
Train: Epoch 0 [7.04] Loss 0.15 [0.14]
Train: Epoch 0 [8.81] Loss 0.09 [0.13]
Train: Epoch 0 [10.57] Loss 0.12 [0.12]
Train: Epoch 0 [12.33] Loss 0.10 [0.12]
Train: Epoch 0 [14.09] Loss 0.12 [0.11]
Train: Epoch 0 [15.85] Loss 0.10 [0.11]
Train: Epoch 0 [17.61] Loss 0.13 [0.11]
Train: Epoch 0 [19.37] Loss 0.10 [0.11]
Train: Epoch 0 [21.13] Loss 0.13 [0.11]
Train: Epoch 0 [22.90] Loss 0.10 [0.11]
Train: Epoch 0 [24.66] Loss 0.09 [0.11]
Train: Epoch 0 [26.42] Loss 0.12 [0.11]
Train: Epoch 0 [28.18] Loss 0.11 [0.10]
Train: Epoch 0 [29.94] Loss 0.07 [0.10]
Train: Epoch 0 [31.70] Loss 0.11 [0.10]
Train: Epoch 0 [33.46] Loss 0.09 [0.09]
Train: Epoch 0 [35.22] Loss 0.11 [0.09]
Train: Epoch 0 [36.98] Loss 0.10 [0.09]
Train: Epoch 0 [38.75] Loss 0.08 [0.09]
Train: Epoch 0 [40.51] Loss 0.09 [0.09]
Train: Epoch 0 [42.27] Loss 0.11 [0.09]
Train: Epoch 0 [44.03] Loss 0.09 [0.09]
Train


Test: Epoch 0 Loss 0.10
Train: Epoch 1 [1.76] Loss 0.09 [0.09]
Train: Epoch 1 [3.52] Loss 0.08 [0.09]
Train: Epoch 1 [5.28] Loss 0.10 [0.08]
Train: Epoch 1 [7.04] Loss 0.08 [0.08]
Train: Epoch 1 [8.81] Loss 0.07 [0.08]
Train: Epoch 1 [10.57] Loss 0.07 [0.08]
Train: Epoch 1 [12.33] Loss 0.08 [0.08]
Train: Epoch 1 [14.09] Loss 0.08 [0.08]
Train: Epoch 1 [15.85] Loss 0.08 [0.08]
Train: Epoch 1 [17.61] Loss 0.10 [0.08]
Train: Epoch 1 [19.37] Loss 0.07 [0.08]
Train: Epoch 1 [21.13] Loss 0.07 [0.08]
Train: Epoch 1 [22.90] Loss 0.07 [0.08]
Train: Epoch 1 [24.66] Loss 0.09 [0.08]
Train: Epoch 1 [26.42] Loss 0.08 [0.08]
Train: Epoch 1 [28.18] Loss 0.09 [0.09]
Train: Epoch 1 [29.94] Loss 0.08 [0.08]
Train: Epoch 1 [31.70] Loss 0.07 [0.08]
Train: Epoch 1 [33.46] Loss 0.07 [0.08]
Train: Epoch 1 [35.22] Loss 0.08 [0.08]
Train: Epoch 1 [36.98] Loss 0.07 [0.08]
Train: Epoch 1 [38.75] Loss 0.08 [0.08]
Train: Epoch 1 [40.51] Loss 0.10 [0.08]
Train: Epoch 1 [42.27] Loss 0.07 [0.08]
Train: Epoch 1 [44.0


Test: Epoch 1 Loss 0.11
Train: Epoch 2 [1.76] Loss 0.10 [0.07]
Train: Epoch 2 [3.52] Loss 0.07 [0.07]
Train: Epoch 2 [5.28] Loss 0.07 [0.07]
Train: Epoch 2 [7.04] Loss 0.07 [0.07]
Train: Epoch 2 [8.81] Loss 0.07 [0.07]
Train: Epoch 2 [10.57] Loss 0.09 [0.07]
Train: Epoch 2 [12.33] Loss 0.07 [0.07]
Train: Epoch 2 [14.09] Loss 0.06 [0.07]
Train: Epoch 2 [15.85] Loss 0.06 [0.07]
Train: Epoch 2 [17.61] Loss 0.07 [0.07]
Train: Epoch 2 [19.37] Loss 0.06 [0.07]
Train: Epoch 2 [21.13] Loss 0.08 [0.07]
Train: Epoch 2 [22.90] Loss 0.07 [0.07]
Train: Epoch 2 [24.66] Loss 0.09 [0.07]
Train: Epoch 2 [26.42] Loss 0.06 [0.07]
Train: Epoch 2 [28.18] Loss 0.08 [0.07]
Train: Epoch 2 [29.94] Loss 0.07 [0.07]
Train: Epoch 2 [31.70] Loss 0.07 [0.07]
Train: Epoch 2 [33.46] Loss 0.07 [0.07]
Train: Epoch 2 [35.22] Loss 0.07 [0.07]
Train: Epoch 2 [36.98] Loss 0.07 [0.07]
Train: Epoch 2 [38.75] Loss 0.07 [0.07]
Train: Epoch 2 [40.51] Loss 0.07 [0.07]
Train: Epoch 2 [42.27] Loss 0.06 [0.07]
Train: Epoch 2 [44.0


Test: Epoch 2 Loss 0.07
Train: Epoch 3 [1.76] Loss 0.07 [0.07]
Train: Epoch 3 [3.52] Loss 0.08 [0.07]
Train: Epoch 3 [5.28] Loss 0.06 [0.07]
Train: Epoch 3 [7.04] Loss 0.08 [0.07]
Train: Epoch 3 [8.81] Loss 0.08 [0.07]
Train: Epoch 3 [10.57] Loss 0.06 [0.07]
Train: Epoch 3 [12.33] Loss 0.07 [0.07]
Train: Epoch 3 [14.09] Loss 0.08 [0.07]
Train: Epoch 3 [15.85] Loss 0.05 [0.07]
Train: Epoch 3 [17.61] Loss 0.07 [0.07]
Train: Epoch 3 [19.37] Loss 0.07 [0.07]
Train: Epoch 3 [21.13] Loss 0.08 [0.07]
Train: Epoch 3 [22.90] Loss 0.07 [0.07]
Train: Epoch 3 [24.66] Loss 0.07 [0.07]
Train: Epoch 3 [26.42] Loss 0.07 [0.07]
Train: Epoch 3 [28.18] Loss 0.05 [0.07]
Train: Epoch 3 [29.94] Loss 0.07 [0.07]
Train: Epoch 3 [31.70] Loss 0.06 [0.07]
Train: Epoch 3 [33.46] Loss 0.08 [0.07]
Train: Epoch 3 [35.22] Loss 0.07 [0.07]
Train: Epoch 3 [36.98] Loss 0.06 [0.07]
Train: Epoch 3 [38.75] Loss 0.06 [0.07]
Train: Epoch 3 [40.51] Loss 0.08 [0.07]
Train: Epoch 3 [42.27] Loss 0.07 [0.07]
Train: Epoch 3 [44.0

...


Test: Epoch 10 Loss 0.06
Train: Epoch 11 [1.76] Loss 0.07 [0.06]
Train: Epoch 11 [3.52] Loss 0.06 [0.06]
Train: Epoch 11 [5.28] Loss 0.06 [0.06]
Train: Epoch 11 [7.04] Loss 0.06 [0.06]
Train: Epoch 11 [8.81] Loss 0.06 [0.06]
Train: Epoch 11 [10.57] Loss 0.05 [0.06]
Train: Epoch 11 [12.33] Loss 0.07 [0.06]
Train: Epoch 11 [14.09] Loss 0.06 [0.06]
Train: Epoch 11 [15.85] Loss 0.05 [0.06]
Train: Epoch 11 [17.61] Loss 0.06 [0.06]
Train: Epoch 11 [19.37] Loss 0.06 [0.06]
Train: Epoch 11 [21.13] Loss 0.07 [0.06]
Train: Epoch 11 [22.90] Loss 0.07 [0.06]
Train: Epoch 11 [24.66] Loss 0.05 [0.06]
Train: Epoch 11 [26.42] Loss 0.07 [0.06]
Train: Epoch 11 [28.18] Loss 0.06 [0.06]
Train: Epoch 11 [29.94] Loss 0.05 [0.06]
Train: Epoch 11 [31.70] Loss 0.05 [0.06]
Train: Epoch 11 [33.46] Loss 0.05 [0.06]
Train: Epoch 11 [35.22] Loss 0.07 [0.06]
Train: Epoch 11 [36.98] Loss 0.06 [0.06]
Train: Epoch 11 [38.75] Loss 0.07 [0.06]
Train: Epoch 11 [40.51] Loss 0.05 [0.06]
Train: Epoch 11 [42.27] Loss 0.06 [0


Test: Epoch 11 Loss 0.06
Train: Epoch 12 [1.76] Loss 0.07 [0.06]
Train: Epoch 12 [3.52] Loss 0.06 [0.06]
Train: Epoch 12 [5.28] Loss 0.06 [0.06]
Train: Epoch 12 [7.04] Loss 0.06 [0.06]
Train: Epoch 12 [8.81] Loss 0.06 [0.06]
Train: Epoch 12 [10.57] Loss 0.08 [0.06]
Train: Epoch 12 [12.33] Loss 0.06 [0.06]
Train: Epoch 12 [14.09] Loss 0.07 [0.06]
Train: Epoch 12 [15.85] Loss 0.07 [0.06]
Train: Epoch 12 [17.61] Loss 0.07 [0.06]
Train: Epoch 12 [19.37] Loss 0.07 [0.06]
Train: Epoch 12 [21.13] Loss 0.06 [0.06]
Train: Epoch 12 [22.90] Loss 0.06 [0.06]
Train: Epoch 12 [24.66] Loss 0.07 [0.06]
Train: Epoch 12 [26.42] Loss 0.06 [0.06]
Train: Epoch 12 [28.18] Loss 0.07 [0.06]
Train: Epoch 12 [29.94] Loss 0.09 [0.06]
Train: Epoch 12 [31.70] Loss 0.06 [0.06]
Train: Epoch 12 [33.46] Loss 0.06 [0.06]
Train: Epoch 12 [35.22] Loss 0.06 [0.06]
Train: Epoch 12 [36.98] Loss 0.05 [0.06]
Train: Epoch 12 [38.75] Loss 0.07 [0.06]
Train: Epoch 12 [40.51] Loss 0.06 [0.06]
Train: Epoch 12 [42.27] Loss 0.06 [0


Test: Epoch 12 Loss 0.06
Train: Epoch 13 [1.76] Loss 0.07 [0.06]
Train: Epoch 13 [3.52] Loss 0.08 [0.06]
Train: Epoch 13 [5.28] Loss 0.06 [0.06]
Train: Epoch 13 [7.04] Loss 0.06 [0.06]
Train: Epoch 13 [8.81] Loss 0.06 [0.06]
Train: Epoch 13 [10.57] Loss 0.07 [0.06]
Train: Epoch 13 [12.33] Loss 0.06 [0.06]
Train: Epoch 13 [14.09] Loss 0.06 [0.06]
Train: Epoch 13 [15.85] Loss 0.07 [0.06]
Train: Epoch 13 [17.61] Loss 0.05 [0.06]
Train: Epoch 13 [19.37] Loss 0.05 [0.06]
Train: Epoch 13 [21.13] Loss 0.06 [0.06]
Train: Epoch 13 [22.90] Loss 0.07 [0.06]
Train: Epoch 13 [24.66] Loss 0.06 [0.06]
Train: Epoch 13 [26.42] Loss 0.06 [0.06]
Train: Epoch 13 [28.18] Loss 0.06 [0.06]
Train: Epoch 13 [29.94] Loss 0.05 [0.06]
Train: Epoch 13 [31.70] Loss 0.06 [0.06]
Train: Epoch 13 [33.46] Loss 0.07 [0.06]
Train: Epoch 13 [35.22] Loss 0.06 [0.06]
Train: Epoch 13 [36.98] Loss 0.07 [0.06]
Train: Epoch 13 [38.75] Loss 0.08 [0.06]
Train: Epoch 13 [40.51] Loss 0.07 [0.06]
Train: Epoch 13 [42.27] Loss 0.08 [0


Test: Epoch 13 Loss 0.06
Train: Epoch 14 [1.76] Loss 0.06 [0.06]
Train: Epoch 14 [3.52] Loss 0.06 [0.06]
Train: Epoch 14 [5.28] Loss 0.06 [0.06]
Train: Epoch 14 [7.04] Loss 0.07 [0.06]
Train: Epoch 14 [8.81] Loss 0.08 [0.06]
Train: Epoch 14 [10.57] Loss 0.06 [0.06]
Train: Epoch 14 [12.33] Loss 0.06 [0.06]
Train: Epoch 14 [14.09] Loss 0.05 [0.06]
Train: Epoch 14 [15.85] Loss 0.06 [0.06]
Train: Epoch 14 [17.61] Loss 0.06 [0.06]
Train: Epoch 14 [19.37] Loss 0.06 [0.06]
Train: Epoch 14 [21.13] Loss 0.08 [0.06]
Train: Epoch 14 [22.90] Loss 0.06 [0.06]
Train: Epoch 14 [24.66] Loss 0.08 [0.06]
Train: Epoch 14 [26.42] Loss 0.07 [0.06]
Train: Epoch 14 [28.18] Loss 0.06 [0.06]
Train: Epoch 14 [29.94] Loss 0.07 [0.06]
Train: Epoch 14 [31.70] Loss 0.07 [0.06]
Train: Epoch 14 [33.46] Loss 0.07 [0.06]
Train: Epoch 14 [35.22] Loss 0.07 [0.06]
Train: Epoch 14 [36.98] Loss 0.05 [0.06]
Train: Epoch 14 [38.75] Loss 0.08 [0.06]
Train: Epoch 14 [40.51] Loss 0.07 [0.06]
Train: Epoch 14 [42.27] Loss 0.09 [0


Test: Epoch 14 Loss 0.06


In [85]:
idx = 0
for data in tqdm(train_dataloader):
    x, y = data
    targets = y[:, -1, :]
    y = y[:, :-1, :]
#     x = x.view(x.size(0), -1)
    x = x.to(device).float()
    y = y.to(device).float()
    targets = targets.to(device).float()
    output, fcn_output, lstm_output = model(x)
    print(output[idx])
    print(fcn_output[idx])
    print(lstm_output[idx])
    print(targets[idx])
    break

tensor([0.2088, 0.8464, 0.4924, 0.7348, 0.1261, 0.0778, 0.0409, 0.0368, 0.1575,
        0.5876, 0.5184, 0.2014, 0.3117, 0.7780, 0.1048, 0.0611, 0.0931, 0.5933,
        0.3245, 0.3218, 0.7190, 0.7837, 0.2320, 0.9537, 0.5250, 0.3488, 0.1724,
        0.2696, 0.2439], grad_fn=<SelectBackward>)
tensor([0.4506, 0.4940, 0.3752, 0.4419, 0.1019, 0.0303, 0.0186, 0.0000, 0.2020,
        0.3642, 0.0000, 0.2073, 0.2147, 0.5529, 0.1413, 0.1206, 0.0000, 0.4793,
        0.2421, 0.2569, 0.4529, 0.5453, 0.2005, 0.6409, 0.3424, 0.2330, 0.1298,
        0.1619, 0.2116], grad_fn=<SelectBackward>)
tensor([0.2063, 0.8228, 0.4972, 0.7060, 0.1373, 0.0817, 0.0480, 0.0376, 0.1169,
        0.5559, 0.4686, 0.1565, 0.3028, 0.7253, 0.1188, 0.0565, 0.0927, 0.5986,
        0.3218, 0.3160, 0.7079, 0.7675, 0.1968, 0.9456, 0.4843, 0.3289, 0.1524,
        0.2542, 0.2254], grad_fn=<SelectBackward>)
tensor([0.1649, 0.8073, 0.5705, 0.7835, 0.1018, 0.0719, 0.0325, 0.0297, 0.1177,
        0.5087, 0.4570, 0.1912, 0.2237, 0.5402,

In [86]:
torch.save(model.state_dict(), 'ensemble2.pth')

In [94]:
d = fish_time = {
    'щука': {
        0: 0.1,
        3: 0.5,
        6: 1.2,
        9: 1.1,
        12: 1.0,
        15: 0.9,
        18: 1.15,
        21: 1.0,
    },
    'судак': {
        0: 1.1,
        3: 1.0,
        6: 1.1,
        9: 1.0,
        12: 1.0,
        15: 0.9,
        18: 1.0,
        21: 1.1,
    },
    'окунь': {
        0: 0.1,
        3: 0.5,
        6: 1.2,
        9: 1.1,
        12: 1.0,
        15: 1.0,
        18: 1.0,
        21: 0.9,
    },
    'берш': {
        0: 1.1,
        3: 1.0,
        6: 1.1,
        9: 1.0,
        12: 1.0,
        15: 0.9,
        18: 1.0,
        21: 1.1,
    },
    'речная форель': {
        0: 0.1,
        3: 0.1,
        6: 1.1,
        9: 1.0,
        12: 1.0,
        15: 0.9,
        18: 0.8,
        21: 0.5,
    },
    'озерная форель': {
        0: 0.1,
        3: 0.1,
        6: 1.1,
        9: 1.0,
        12: 1.0,
        15: 0.9,
        18: 0.8,
        21: 0.5,
    },
    'елец': {
        0: 0.1,
        3: 0.1,
        6: 0.9,
        9: 1.0,
        12: 1.1,
        15: 1.1,
        18: 0.8,
        21: 0.3,
    },
    'чехонь': {
        0: 0.1,
        3: 0.1,
        6: 1.1,
        9: 1.0,
        12: 1.1,
        15: 1.0,
        18: 0.7,
        21: 0.3,
    },
    'сом': {
        0: 1.1,
        3: 1.2,
        6: 0.8,
        9: 0.6,
        12: 0.5,
        15: 0.4,
        18: 0.6,
        21: 0.9,
    },
    'голавль': {
        0: 0.7,
        3: 0.7,
        6: 1.1,
        9: 1.0,
        12: 1.0,
        15: 1.0,
        18: 0.8,
        21: 0.7,
    },
    'язь': {
        0: 0.9,
        3: 0.9,
        6: 1.1,
        9: 1.0,
        12: 1.0,
        15: 1.0,
        18: 1.1,
        21: 0.9,
    },
    'карп': {
        0: 0.9,
        3: 0.9,
        6: 1.1,
        9: 1.0,
        12: 1.0,
        15: 1.0,
        18: 1.1,
        21: 0.9,
    },
    'жерех': {
        0: 0.3,
        3: 0.5,
        6: 1.1,
        9: 1.0,
        12: 1.0,
        15: 1.0,
        18: 1.1,
        21: 0.7,
    },
    'лещ': {
        0: 1.1,
        3: 1.1,
        6: 1.0,
        9: 0.9,
        12: 0.8,
        15: 0.8,
        18: 1.0,
        21: 1.1,
    },
    'карась': {
        0: 0.1,
        3: 0.4,
        6: 1.1,
        9: 1.1,
        12: 0.9,
        15: 0.9,
        18: 1.0,
        21: 0.6,
    },
    'линь': {
        0: 0.1,
        3: 0.4,
        6: 1.1,
        9: 1.1,
        12: 0.9,
        15: 0.9,
        18: 1.0,
        21: 0.6,
    },
    'пескарь': {
        0: 0.1,
        3: 0.4,
        6: 0.8,
        9: 1.0,
        12: 1.1,
        15: 1.1,
        18: 1.0,
        21: 0.6,
    },
    'ротан': {
        0: 0.5,
        3: 0.7,
        6: 1.1,
        9: 1.1,
        12: 1.0,
        15: 1.0,
        18: 1.1,
        21: 0.9,
    },
    'плотва': {
        0: 0.2,
        3: 0.5,
        6: 1.1,
        9: 1.1,
        12: 1.0,
        15: 1.0,
        18: 1.1,
        21: 0.6,
    },
    'красноперка': {
        0: 0.2,
        3: 0.5,
        6: 1.1,
        9: 1.1,
        12: 1.0,
        15: 1.0,
        18: 1.1,
        21: 0.6,
    },
    'налим': {
        0: 1.1,
        3: 1.1,
        6: 1.0,
        9: 0.8,
        12: 0.4,
        15: 0.6,
        18: 0.9,
        21: 1.0,
    },
    'густера': {
        0: 0.8,
        3: 0.9,
        6: 1.1,
        9: 1.1,
        12: 1.0,
        15: 1.0,
        18: 1.1,
        21: 0.9,
    },
    'амур': {
        0: 0.8,
        3: 0.9,
        6: 1.1,
        9: 1.1,
        12: 1.0,
        15: 1.0,
        18: 1.1,
        21: 1.1,
    },
    'ерш': {
        0: 1.1,
        3: 1.1,
        6: 1.0,
        9: 0.8,
        12: 0.7,
        15: 0.7,
        18: 0.9,
        21: 1.0,
    },
    'сазан': {
        0: 0.8,
        3: 0.9,
        6: 1.1,
        9: 1.1,
        12: 1.0,
        15: 1.0,
        18: 1.1,
        21: 0.9,
    },
    'подуст': {
        0: 0.6,
        3: 0.7,
        6: 1.1,
        9: 1.1,
        12: 0.9,
        15: 0.9,
        18: 0.8,
        21: 0.7,
    },
    'толстолобик': {
        0: 0.4,
        3: 0.5,
        6: 1.1,
        9: 1.1,
        12: 0.9,
        15: 0.9,
        18: 0.7,
        21: 0.5,
    },
    'вобла': {
        0: 0.3,
        3: 0.4,
        6: 1.1,
        9: 1.1,
        12: 1.0,
        15: 1.0,
        18: 0.9,
        21: 0.6,
    },
    'хариус': {
        0: 0.3,
        3: 0.4,
        6: 1.1,
        9: 1.1,
        12: 1.0,
        15: 1.0,
        18: 0.9,
        21: 0.6,
    }
}

In [95]:
for k in d:
    d[k] = list(d[k].values())

In [96]:
d_ = {}
for k in d:
    d_[k.capitalize()] = d[k]
#     del d[k]

In [97]:
d_

{'Щука': [0.1, 0.5, 1.2, 1.1, 1.0, 0.9, 1.15, 1.0],
 'Судак': [1.1, 1.0, 1.1, 1.0, 1.0, 0.9, 1.0, 1.1],
 'Окунь': [0.1, 0.5, 1.2, 1.1, 1.0, 1.0, 1.0, 0.9],
 'Берш': [1.1, 1.0, 1.1, 1.0, 1.0, 0.9, 1.0, 1.1],
 'Речная форель': [0.1, 0.1, 1.1, 1.0, 1.0, 0.9, 0.8, 0.5],
 'Озерная форель': [0.1, 0.1, 1.1, 1.0, 1.0, 0.9, 0.8, 0.5],
 'Елец': [0.1, 0.1, 0.9, 1.0, 1.1, 1.1, 0.8, 0.3],
 'Чехонь': [0.1, 0.1, 1.1, 1.0, 1.1, 1.0, 0.7, 0.3],
 'Сом': [1.1, 1.2, 0.8, 0.6, 0.5, 0.4, 0.6, 0.9],
 'Голавль': [0.7, 0.7, 1.1, 1.0, 1.0, 1.0, 0.8, 0.7],
 'Язь': [0.9, 0.9, 1.1, 1.0, 1.0, 1.0, 1.1, 0.9],
 'Карп': [0.9, 0.9, 1.1, 1.0, 1.0, 1.0, 1.1, 0.9],
 'Жерех': [0.3, 0.5, 1.1, 1.0, 1.0, 1.0, 1.1, 0.7],
 'Лещ': [1.1, 1.1, 1.0, 0.9, 0.8, 0.8, 1.0, 1.1],
 'Карась': [0.1, 0.4, 1.1, 1.1, 0.9, 0.9, 1.0, 0.6],
 'Линь': [0.1, 0.4, 1.1, 1.1, 0.9, 0.9, 1.0, 0.6],
 'Пескарь': [0.1, 0.4, 0.8, 1.0, 1.1, 1.1, 1.0, 0.6],
 'Ротан': [0.5, 0.7, 1.1, 1.1, 1.0, 1.0, 1.1, 0.9],
 'Плотва': [0.2, 0.5, 1.1, 1.1, 1.0, 1.0, 1.1, 0.6]